In [1]:
import numpy as np
import pandas as pd
import re
from os.path import join

from tools import *
from S2.utils._path import *

from sklearn.decomposition import LatentDirichletAllocation
from scipy.spatial import distance

import matplotlib.pyplot as plt
import numpy as np

## rename EXTRACT_FEATURES to DIMENSION_REDUCTION
read_prefix = "/data/MIMIC3/"
write_prefix = "/data/liu/LDA"
res_prefix = "/data/liu/LDA/lda_result"
res_r_prefix = "/data/liu/LDA/lda_R_result/"

In [27]:
# from S2.cluster_analysis import feature_creation
# fc = feature_creation()
# diag_matrix = fc.create_data()

## Create Feature representatives combien five tables

## diagnosis, prescription, procedure

In [28]:
# ## sampled hadmid
# sample_epis_file='HADM_ID_SAMPLE_PER_PATIENT'
# sampled_hadmid = read_data(join(singledrug_prefix,sample_epis_file),dtype=str)
# sampled_hadmid.head()

In [17]:
def print_statistics(df):
    
    print("# of rows: %d"%len(df))
    print("# of patients: %d"%len(df['SUBJECT_ID'].unique()))
    print("# of episodes: %d"%len(df['HADM_ID'].unique()))
    
def df2matrix(df):
    cols=["",df.columns[-1]]
    print("Original Data:")
    print_statistics(df)
    df_sampled = left_join(sampled_hadmid,df,list(sampled_hadmid.columns))
    print("Sampled Data:")
    print_statistics(df_sampled)
    
    
    
    # pres_matrix = pres_df_1k_1k_hadm.pivot(index='SUBJECT_ID', columns='NDC', values='VALUE').fillna(0).reset_index()

    return df_sampled

In [29]:
## diag_df 
## ICD9_CODE, 1
# diag_df = read_data(join(read_prefix,"DIAGNOSES_ICD"),dtype={"SUBJECT_ID":str,"HADM_ID":str})
# diag_df.head()
# df2matrix(diag_df,"aa")

In [30]:
# ## presdf
# pres_df = read_data(
#     join(read_prefix,'PRESCRIPTIONS'),
#     dtype={"SUBJECT_ID":str,"HADM_ID":str,"NDC":str})[["SUBJECT_ID","HADM_ID","NDC"]].dropna(subset=['NDC'])
# pres_df = pres_df[pres_df['NDC']!="0"]
# # read_data(join(read_prefix,"DIAGNOSES_ICD"),dtype={"SUBJECT_ID":str,"HADM_ID":str})
# pres_df.head()
# # df2matrix(pres_df)

## LABEVENTS

In [2]:
# ITEMID, VALUE
## remove rows with none value in episodes(keep coherent to other table with episodes)
lab_df=read_data(
    join(read_prefix,'LABEVENTS'),
    dtype={"SUBJECT_ID":str,"HADM_ID":str})[["SUBJECT_ID","HADM_ID","ITEMID","VALUE"]].dropna(subset=['HADM_ID','VALUE'])

# df2matrix(lab_df,"aa")

In [3]:
remove_value=['.']
lab_df=lab_df.loc[~lab_df.VALUE.isin(remove_value)]

In [ ]:
lab_df.head()

In [8]:
def is_number(s):
    try:
        float(s.strip())
        return True
    except ValueError:
        return False
    
def process_df_value_col(df):
# temp_df.groupby(["HADM_ID","ITEMID"])["VALUE"].mean()
    cols=df.columns
    df["real"] = df["VALUE"].apply(is_number)
    print(len(df[df["real"]][cols[2]].unique()))
    print(len(df[~df["real"]][cols[2]].unique()))

# .apply(lambda s: pd.to_numeric(s, errors='coerce'))
    return df.groupby(cols[2])["real"].agg(all)


In [24]:
test = process_df_value_col(lab_df)
test

/home/liu/anaconda3/envs/tfgpu21/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


480
482


ITEMID
50800    False
50801     True
50802    False
50803    False
50804    False
         ...  
51520    False
51523    False
51529     True
51532     True
51537    False
Name: real, Length: 710, dtype: bool

In [29]:
with open(join("S2_output","DimensionReduction_LABEVENTS_mixOF_discreteContinuous_value.txt"), "a") as f:
    for itemid in test[~test].index:
        values = lab_df[lab_df['ITEMID']==itemid]["VALUE"].unique()
        if(len(values)>10):
            print("ITEMID=%d"%itemid,file=f)
            print("%s\n"%values,file=f)

In [14]:
test2 = lab_df[lab_df['ITEMID']==50878]

In [15]:
test2['real']=test2['VALUE'].apply(is_number)
test2[~test2['real']]

/home/liu/anaconda3/envs/tfgpu21/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,SUBJECT_ID,HADM_ID,ITEMID,VALUE,real
62999,117,164853,50878,ERROR,False
3533515,5568,198198,50878,QNS,False
7512002,12062,167804,50878,GREATER THAN 800,False
7956366,12720,175987,50878,"DISREGARD PREVIOUS RESULT 32,SPECIMEN MISLABELLED",False
8068055,12756,194776,50878,HEMOLYSIS FALSELY INCREASES THIS RESULT,False
19441092,32476,119862,50878,VERIFIED BY DILUTION,False
21275709,54586,122570,50878,ERROR,False
27196030,91877,157472,50878,ADD-ON QNS,False


In [41]:
is_number(test2.loc[5164525,"VALUE"])

True

In [24]:
## procedures
procedure_df=read_data(
    join(read_prefix,'PROCEDURES_ICD'),
    dtype={"SUBJECT_ID":str,"HADM_ID":str})
procedure_df.head()


,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,944,62641,154460,3,3404
1,945,2592,130856,1,9671
2,946,2592,130856,2,3893
3,947,55357,119355,1,9672
4,948,55357,119355,2,331


## CHECK TEXT

In [21]:
from IPython.display import HTML
HTML(filename='/data/risa/annotate/annotate_9981_125955_1.html')